# Proyecto final de Audio

In [ ]:
from google.colab import userdata

fpath = '/content/audiopfinal/data'


token = userdata.get('ghToken')
!git clone https://{token}@github.com/Cosuki/audiopfinal.git > /dev/null 2>&1


In [ ]:

import pandas as pd
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
from scipy import io, signal
from scipy.io.wavfile import read
from IPython.display import Audio
import scipy.io.wavfile as wav
from matplotlib.pyplot import specgram
from scipy.signal import spectrogram

fpath = './data'

def pltlegacy(ax, title, xLabel, yLabel):
  if type(ax) == np.ndarray or type(ax) == plt.Axes:
    ax.set_title(title)
    # ax.set_ylim(top=10000)
    ax.set_xlabel(xLabel)
    ax.set_ylabel(yLabel)
    if not yLabel: ax.get_yaxis().set_visible(False)
    # if not xLabel: ax.get_xaxis().set_visible(False)
  else:
    ax.title(title)
    # ax.set_ylim(top=10000)
    ax.xlabel(xLabel)
    ax.ylabel(yLabel)

def getpltmtplt(ax, key, data, sr, xLabel, yLabel):
    Pxx, freqs, bins, im = ax.specgram(data, NFFT=1024, srG=sr, noverlap=128, cmap='viridis')
    pltlegacy(ax, f'{key}', xLabel, yLabel)
    return Pxx, freqs, bins, im

def getSpecSig(ax, key, data, sr, xLabel, yLabel):
  f, t_spec, Sxx = spectrogram(data, sr)
  Sxx += 1e-10
  pltlegacy(ax, f'{key}', xLabel, yLabel)
  return ax.pcolormesh(t_spec, f, 10 * np.log10(Sxx), shading='gouraud')

In [ ]:
srG, guitar = io.wavfile.read(f'{fpath}/Police-vocals-guitar.wav')
srV, voice = io.wavfile.read(f'{fpath}/Emily_Linge-vocals.wav')

print(f'{srG=}, {srV=}')

guitar = guitar[0: 30*srG]
voice = voice[0: 30*srV]

display(Audio(guitar, rate=srG), Audio(voice, rate=srV))

In [ ]:
from scipy.linalg import toeplitz, solve
import numpy as np

def get_lpc(s, p=20):
    """Compute the LPC analysis using the autocorrelation method."""
    N = s.shape[0]
    p = min(p, N - 1)  # Asegurar que p no sea mayor que el tamaño del marco

    # Compute autocorrelation values
    r = np.zeros((p + 1,))
    for k in range(p + 1):
        r[k] = np.dot(s[:N - k], s[k:])

    if np.all(r == 0):
        raise ValueError("Autocorrelation is zero; the input signal may be silent or constant.")

    # Solve to compute model coefficients
    try:
        ak = solve(toeplitz(r[:p]) + 1e-6 * np.eye(p), r[1:p + 1]).squeeze()
    except np.linalg.LinAlgError:
        raise ValueError("Matrix is singular. Adjust the model order or check input data.")

    # Compute mean squared error
    e = r[0] - np.dot(ak.T, r[1:p + 1])

    # Compute normalized mean squared error
    e_norm = e / r[0] if r[0] != 0 else 0

    return ak, e, e_norm


In [ ]:
from typing import Any


from numpy import dtype
from numpy._typing._array_like import NDArray


def lpc_analysis(sig, sr, t, getPlot=False, p=0, retType='dB', Ndft=1024):
    n = int(t * sr)
    window = signal.windows.get_window('hann', n)  # Smoothing window
    lpc_matrix = []
    threshold = 1e-12  # Umbral para detectar silencio
    
    i = 0
    for chunk in np.array_split(sig, int(len(sig) / n)):
        i += 1
        if len(chunk) < n:
            chunk = np.pad(chunk, (0, n - len(chunk)), mode='constant')  # Padding
        
        window = signal.windows.get_window('hann', len(chunk))  # Ajustar ventana
        s_win = chunk * window  # Señal con ventana
        
        # Detectar silencio
        if np.max(np.abs(s_win)) < threshold:
            # print(f"Skipping silent segment {i}")
            lpc_matrix.append(np.zeros(Ndft // 2))
            continue
        
        # Normalizar segmento
        s_win = s_win / (np.max(np.abs(s_win)) + 1e-12)
        
        try:
            ak, e, e_norm = get_lpc(s_win, p if p else int(sr / 1000))
        except ValueError as ve:
            print(f"Error processing segment: {ve}")
            continue
        
        
        # Espectro de magnitud
        X = np.fft.fft(s_win, Ndft)
        magX = np.abs(X[:Ndft // 2])
        
        # filter obtained from the lpc analysis
        S = 1
        U = np.concatenate([[1], -ak])
        G = np.sqrt(e) # compute gain
        w, H = signal.freqz(G*S, U, worN=Ndft, whole=True) # compute the frequency response of the digital filter
        magH = np.abs(H[:Ndft // 2])
        
        lpc_matrix.append(20 * np.log10(magH) if retType == 'dB' else magX)
        
        if getPlot:
            plt.plot(20 * np.log10(magX))
            plt.title("Magnitude Spectrum")
            plt.show()

    
    return np.array(lpc_matrix)


In [ ]:

t = .02

a = lpc_analysis(guitar, srG, t)
b = lpc_analysis(voice, srV, t)

In [ ]:
np.arange(stop=100, step=4)
print(a.shape, b.shape)
for i in np.arange(stop=3000, step=5):
    fig, axs = plt.subplots(1, 5, figsize=(25, 4))
    axs[0].plot(a[i])
    axs[0].plot(b[i])
    axs[1].plot(a[i+1])
    axs[1].plot(b[i+1])
    axs[2].plot(a[i+2])
    axs[2].plot(b[i+2])
    axs[3].plot(a[i+3])
    axs[3].plot(b[i+3])
    axs[4].plot(a[i+4])
    axs[4].plot(b[i+4])
    plt.show()

In [ ]:
import numpy as np
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean
import matplotlib.pyplot as plt

# Supongamos que estas son las matrices LPC de las dos señales
# lpc_matrix1: Matriz de LPC de la primera señal (N1 x D)
# lpc_matrix2: Matriz de LPC de la segunda señal (N2 x D)

# Calcula la alineación DTW entre las dos matrices
distance, path = fastdtw(a, b, radius=300, dist=euclidean)
# Extraer los índices de alineación
indices1, indices2 = zip(*path)

plt.figure(figsize=(10, 6))
plt.plot(indices1, indices2, 'o-', label='Warping Path', markersize=.1)
plt.xlabel('Tramos de guitar')
plt.ylabel('Tramos de voice')
plt.title('Alineación de DTW')
plt.legend()
plt.grid()
plt.show()



In [ ]:
aligned_signal2 = []

voice_splited = np.array_split(voice, int(len(voice) / int(t * srG)))
for idx1, idx2 in path:
    aligned_signal2.extend(voice_splited[idx2])  # Toma el tramo correspondiente

aligned_signal2 = np.array(aligned_signal2)

print(aligned_signal2.shape)
index = min(len(aligned_signal2), len(guitar))
suma = []
suma.append(guitar[:index])
suma.append(aligned_signal2[:index])
display(Audio(suma, rate=srV))


In [ ]:
import numpy as np
from fastdtw import fastdtw
from scipy.signal import resample

def synchronize_tracks(lpc1, lpc2, signal1, signal2, sr):
    """
    Sincroniza dos pistas de audio usando DTW y matrices LPC.
    
    Parameters
    ----------
    lpc1 : numpy.ndarray
        LPC matrix de la primera señal.
    lpc2 : numpy.ndarray
        LPC matrix de la segunda señal.
    signal1 : numpy.ndarray
        Señal original de la primera pista.
    signal2 : numpy.ndarray
        Señal original de la segunda pista.
    sr : int
        Frecuencia de muestreo de las señales.

    Returns
    -------
    adjusted_signal1 : numpy.ndarray
        Primera señal ajustada a la alineación de la segunda.
    """
    # Calcular DTW entre las matrices LPC
    dist, path = fastdtw(lpc1, lpc2, dist=euclidean)
    print(f"DTW alignment distance: {dist} {np.array(path).shape}")
    
    # Mapeo de alineación
    alignment_1_to_2 = path[0]
    alignment_2_to_1 = path[1]
    
    # Ajustar la primera señal para alinearse con la segunda
    adjusted_signal1 = []
    splited_signal2 = np.array_split(signal2, int(len(signal2) / int(.030 * srG)))
    for idx1, idx2 in zip(alignment_1_to_2, alignment_2_to_1):
        segment = splited_signal2[idx2]
        if len(segment) > 0:
            target_length = int(sr * 0.03)
            adjusted_segment = resample(segment, target_length)
            adjusted_signal1.append(adjusted_segment)
    
    print(np.array(adjusted_signal1).shape)
    
    # Convertir a una señal única
    adjusted_signal1 = np.concatenate(adjusted_signal1)
    
    return adjusted_signal1


In [ ]:
# Asumamos que tienes las matrices LPC y señales originales
adjusted_track = synchronize_tracks(a, b, guitar, voice, srG)



print(adjusted_track.shape)
Audio(adjusted_track, rate=srV)